## Initialize

In [ ]:
import requests
import os
import sys
from copy import deepcopy
import json

sys.path.append(os.path.join(sys.path[0], '..', '..', '..','..'))
print(sys.path[-1])

from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

base_url = get_base_url()
root_path = get_root_path()
example_name = 'dataLoad'

cson_file_direct = os.path.join(root_path, '..','..','..','shared','temp', 'dataLoad.model.json')


if os.path.exists(cson_file_direct):
    with open(cson_file_direct, encoding='utf-8') as f:
        cson = json.load(f)
else:
    cds_file = os.path.join(root_path, '..', 'model.cds')
    cson_file = os.path.join(root_path, '..','..','..', 'shared', 'temp', f'dataLoad.model.json')
    command_line_statement = (f'cds compile {cds_file} --to json -o {cson_file}')
    os.system(command_line_statement)
    with open(cson_file, encoding='utf-8') as f:
        cson = json.load(f)
    print(f'The following command line statement was executed to compile the example cds file to cson. Filename is temp{os.sep}dataLoad.model.json):')
    print(command_line_statement)



## Test data
- Person with and without sourceId
- Organization with and without sourceId
- Relation between person and organization

In [ ]:
person = {
    'firstName': 'John',
    'lastName': 'Doe'
}
person_src = deepcopy(person) | {
    'source': [
        {
            'name': 'systemA',
            'type': 'Person',
            'sid': 'objectid123456'
        }
    ]
}

organization = {
    'name': 'ACME inc.'
}
organization_src = deepcopy(organization) | {
    'source': [
        {
            'name': 'systemA',
            'type': 'Organization',
            'sid': 'objectid098765'
        }
    ]
}

rel_org_person_src = {
    'source': [
        {
            'name': 'systemA',
            'type': 'RelOrgPerson',
            'sid': 'objectid2222444555'
        }
    ],
    'person': {
        'source': [
            {
                'name': 'systemA',
                'type': 'Person',
                'sid': 'objectid123456'
            }
        ]
    },
    'organization': {
        'source': [
            {
                'name': 'systemA',
                'type': 'Organization',
                'sid': 'objectid098765'
            }
        ]
    }
}

## Test __with__ source key
- Preparation of data with prefix "A"
- Deletion of tenant
- Create tenant
- Deploy model
- Uplaod data
- Get ID´s of person and orginzation from data upload request
- Upload same data again, it is expected that status code 400 is returned
- Prepare data for RelOrgPerson relationship and upload it
- Prepare all data with their Id´s (for deletion)
- Delete data

In [ ]:
#### Test 1: A-Objects with source key
object_prefix = 'A'

data = {
    f'{object_prefix}Person': [person_src],
    f'{object_prefix}Organization': [organization_src]
}

r = requests.delete(f'{base_url}/v1/tenant/{example_name}{TENANT_SUFFIX}')
print(f'Delete tenanat status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/tenant/{example_name}{TENANT_SUFFIX}')
print(f'Create tenanat status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/deploy/{example_name}{TENANT_SUFFIX}', json=cson)
print(f'Deploy status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(f'Post data status code: {r.status_code}')
person_id = r.json()[f'{object_prefix}Person'][0]['id']
organization_id = r.json()[f'{object_prefix}Organization'][0]['id']

r = requests.post(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(f'Post data again status code: {r.status_code}. Shoud be 400')

data = {
    f'{object_prefix}RelOrgPerson': [rel_org_person_src]
}
r = requests.post(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(r.status_code)
rel_org_person_id = r.json()[f'{object_prefix}RelOrgPerson'][0]['id']
data = {
    f'{object_prefix}Person': [{'id': person_id}],
    f'{object_prefix}Organization': [{'id': organization_id}],
    f'{object_prefix}RelOrgPerson': [{'id': rel_org_person_id}]
}
r = requests.delete(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(r.status_code)


## Test __without__ source key
- Preparation of data with prefix "A"
- Deletion of tenant
- Create tenant
- Deploy model
- Uplaod data
- Get ID´s of person and orginzation from data upload request
- Upload same data again, it is expected that status code 400 is returned
- Prepare data for RelOrgPerson relationship and upload it
- Prepare all data with their Id´s (for deletion)
- Delete data

In [ ]:
#### Test 2: A-Objects without source key
object_prefix = 'A'
data = {
    f'{object_prefix}Person': [person],
    f'{object_prefix}Organization': [organization]
}

r = requests.delete(f'{base_url}/v1/tenant/{example_name}{TENANT_SUFFIX}')
print(f'Delete tenanat status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/tenant/{example_name}{TENANT_SUFFIX}')
print(f'Create tenanat status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/deploy/{example_name}{TENANT_SUFFIX}', json=cson)
print(f'Deploy status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(r.status_code)
person_id = r.json()[f'{object_prefix}Person'][0]['id']
organization_id = r.json()[f'{object_prefix}Organization'][0]['id']

data = {
    f'{object_prefix}RelOrgPerson': [{'person':{'id':person_id}, 'organization':{'id':organization_id}}]
}
r = requests.post(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(r.status_code)
rel_org_person_id = r.json()[f'{object_prefix}RelOrgPerson'][0]['id']
data = {
    f'{object_prefix}Person': [{'id': person_id}],
    f'{object_prefix}Organization': [{'id': organization_id}],
    f'{object_prefix}RelOrgPerson': [{'id': rel_org_person_id}]
}
r = requests.delete(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(r.status_code)

## Test __with no__ source keys at all
- Preparation of data with prefix "B"
- Deletion of tenant
- Create tenant
- Deploy model
- Uplaod data
- Get ID´s of person and orginzation from data upload request
- Upload same data again, it is expected that status code 400 is returned
- Prepare data for RelOrgPerson relationship and upload it
- Prepare all data with their Id´s (for upload again)
- Delete data

In [ ]:
#### Test 3: B-Objects (they do not have source keys at all)
object_prefix = 'B'
data = {
    f'{object_prefix}Person': [person],
    f'{object_prefix}Organization': [organization]
}

r = requests.delete(f'{base_url}/v1/tenant/{example_name}{TENANT_SUFFIX}')
print(f'Delete tenanat status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/tenant/{example_name}{TENANT_SUFFIX}')
print(f'Create tenanat status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/deploy/{example_name}{TENANT_SUFFIX}', json=cson)
print(f'Deploy status code: {r.status_code}')
r = requests.post(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(r.status_code)
person_id = r.json()[f'{object_prefix}Person'][0]['id']
organization_id = r.json()[f'{object_prefix}Organization'][0]['id']

data = {
    f'{object_prefix}RelOrgPerson': [{'person':{'id':person_id}, 'organization':{'id':organization_id}}]
}
r = requests.post(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(r.status_code)
rel_org_person_id = r.json()[f'{object_prefix}RelOrgPerson'][0]['id']
data = {
    f'{object_prefix}Person': [{'id': person_id}],
    f'{object_prefix}Organization': [{'id': organization_id}],
    f'{object_prefix}RelOrgPerson': [{'id': rel_org_person_id}]
}
r = requests.post(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
print(f'Post data again status code: {r.status_code}. Shoud be 400')
r = requests.delete(f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=data)
